# PDF-Verarbeitung für BundesFAQ-Chatbot

Dieses Notebook demonstriert die PDF-Verarbeitung mit PyMuPDFLoader für den BundesFAQ-Chatbot.

In [36]:
from langchain_community.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from typing import List

In [37]:
print(" PyMuPDFLoader Test")

try:
    pymupdf_loader = PyMuPDFLoader("../data/faq.pdf")
    pymupdf_docs = pymupdf_loader.load()
    
    print(f" {len(pymupdf_docs)} Seiten erfolgreich geladen")
    print(f" Erste Seite (Vorschau): {pymupdf_docs[0].page_content[:100]}...")
    print(f" Metadaten: Autor = {pymupdf_docs[0].metadata.get('author', 'Unbekannt')}")
    
except Exception as e:
    print(f" Fehler: {e}")

 PyMuPDFLoader Test
 22 Seiten erfolgreich geladen
 Erste Seite (Vorschau): Häufig gestellte Fragen
...
 Metadaten: Autor = Bundesverwaltungsamt


## PDF-Verarbeitungsherausforderungen

In [38]:
class SmartPDFProcessor:
    
    def __init__(self, chunk_size=1000, chunk_overlap=100):
        self.chunk_size = chunk_size
        self.chunk_overlap = chunk_overlap
        self.text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=chunk_size,
            chunk_overlap=chunk_overlap,
            separators=["\n\n", "\n", " ", ""],
        )

    def process_pdf(self, pdf_path: str) -> List[Document]:
        """
        Verarbeitet PDF mit intelligenter Chunking und Metadaten-Anreicherung
        
        Args:
            pdf_path: Pfad zur PDF-Datei
            
        Returns:
            Liste von Document-Objekten mit bereinigtem Text und Metadaten
        """
        
        loader = PyMuPDFLoader(pdf_path)
        pages = loader.load()
        
        processed_chunks = []
        
        for page_num, page in enumerate(pages):
            cleaned_text = self._clean_text(page.page_content)
            
            if len(cleaned_text.strip()) < 50:
                continue
            
            chunks = self.text_splitter.create_documents(
                texts=[cleaned_text],
                metadatas=[{
                    **page.metadata,
                    "seite": page_num + 1,
                    "gesamtseiten": len(pages),
                    "chunk_methode": "smart_pdf_processor_pymupdf",
                    "zeichenanzahl": len(cleaned_text),
                    "loader_typ": "PyMuPDFLoader",
                    "sprache": "deutsch"
                }]
            )
            
            processed_chunks.extend(chunks)
        
        return processed_chunks

    def _clean_text(self, text: str) -> str:
        """Bereinigt extrahierten Text für bessere FAQ-Verarbeitung"""
        
        text = " ".join(text.split())
        
        text = text.replace("ﬁ", "fi")
        text = text.replace("ﬂ", "fl")
        text = text.replace("ﬀ", "ff")
        
        text = text.replace("Frage:", "\nFrage:")
        text = text.replace("Antwort:", "\nAntwort:")
        
        return text.strip()

In [39]:
pdf_processor = SmartPDFProcessor(chunk_size=1000, chunk_overlap=100)
print(" SmartPDFProcessor bereit für die Verarbeitung")

 SmartPDFProcessor bereit für die Verarbeitung


In [40]:
print(" Verarbeite die FAQ-PDF-Datei...")

try:
    smart_chunks = pdf_processor.process_pdf("../data/faq.pdf")
    
    print(f" Erfolgreich in {len(smart_chunks)} intelligente Chunks verarbeitet")
    
    if smart_chunks:
        print("\n Beispiel-Chunk-Metadaten:")
        sample_metadata = smart_chunks[0].metadata
        for key, value in sample_metadata.items():
            print(f"  {key}: {value}")
        
        print(f"\n Beispiel-Textinhalt (erste 200 Zeichen):")
        print(f"  {smart_chunks[0].page_content[:200]}...")

except Exception as e:
    print(f" Verarbeitungsfehler: {e}")

 Verarbeite die FAQ-PDF-Datei...
 Erfolgreich in 61 intelligente Chunks verarbeitet

 Beispiel-Chunk-Metadaten:
  source: ../data/faq.pdf
  file_path: ../data/faq.pdf
  page: 1
  total_pages: 22
  format: PDF 1.4
  title: 
  author: Bundesverwaltungsamt
  subject: 
  keywords: 
  creator: 
  producer: 
  creationDate: 
  modDate: 
  trapped: 
  seite: 2
  gesamtseiten: 22
  chunk_methode: smart_pdf_processor_pymupdf
  zeichenanzahl: 442
  loader_typ: PyMuPDFLoader
  sprache: deutsch

 Beispiel-Textinhalt (erste 200 Zeichen):
  Das Open Data Handbuch des BVA FAQ Fragen und Antworten rund um die Umsetzung von Open Data in der Bundesverwaltung Hier finden Sie Antworten zu häufig gestellten Fragen. Diese Übersicht beansprucht n...


In [41]:
smart_chunks

[Document(page_content='Das Open Data Handbuch des BVA FAQ Fragen und Antworten rund um die Umsetzung von Open Data in der Bundesverwaltung Hier finden Sie Antworten zu häufig gestellten Fragen. Diese Übersicht beansprucht nicht die Vollständigkeit und wird im Laufe der Zeit immer wieder ergänzt werden. Ihre Frage wird in diesem FAQ nicht behandelt? Dann schreiben Sie uns eine E-Mail an opendata@bva.bund.de, wir helfen Ihnen gerne weiter. Version 1.5 August 2022', metadata={'source': '../data/faq.pdf', 'file_path': '../data/faq.pdf', 'page': 1, 'total_pages': 22, 'format': 'PDF 1.4', 'title': '', 'author': 'Bundesverwaltungsamt', 'subject': '', 'keywords': '', 'creator': '', 'producer': '', 'creationDate': '', 'modDate': '', 'trapped': '', 'seite': 2, 'gesamtseiten': 22, 'chunk_methode': 'smart_pdf_processor_pymupdf', 'zeichenanzahl': 442, 'loader_typ': 'PyMuPDFLoader', 'sprache': 'deutsch'}),
 Document(page_content='2 Inhaltsverzeichnis Einleitung .....................................